In [ ]:
# default_exp runs

In [ ]:
import sys

In [ ]:
sys.path.append('/workspace/oct_ca_seg/oct/')

In [ ]:
from oct.startup import *
from model import CapsNet
import numpy as np
import mlflow
from fastai.vision import *
import mlflow.pytorch as MLPY
from fastai.utils.mem import gpu_mem_get_all

In [5]:
gpu_mem_get_all()

[GPUMemory(total=16280, free=16270, used=10)]

### Configuration Setup

In [6]:
name = 'DEEPCAP'

In [7]:
config_dict = loadConfigJSONToDict('configCAPS_small.json')
config_dict['LEARNER']['lr']= 0.001
config_dict['LEARNER']['bs'] = 16
config_dict['LEARNER']['epochs'] = 15
config_dict['LEARNER']['runsave_dir'] = '/workspace/oct_ca_seg/runsaves/'
config = DeepConfig(config_dict)

In [8]:
metrics = [sens, spec, dice, my_Dice_Loss, acc]

In [9]:
def saveConfigRun(dictiontary, run_dir, name):
    with open(run_dir/name, 'w') as file:
        json.dump(dictiontary, file)

## Dataset

In [10]:
cocodata_path = Path('/workspace/oct_ca_seg/COCOdata/')
train_path = cocodata_path/'train/images'
valid_path = cocodata_path/'valid/images'
test_path = cocodata_path/'test/images'

### For complete dataset

In [11]:
fn_get_y = lambda image_name: Path(image_name).parent.parent/('labels/'+Path(image_name).name)
codes = np.loadtxt(cocodata_path/'codes.txt', dtype=str)
tfms = get_transforms()
src = (SegCustomItemList
       .from_folder(cocodata_path, recurse=True, extensions='.jpg')
       .filter_by_func(lambda fname: Path(fname).parent.name == 'images', )
       .split_by_folder('train', 'valid')
       .label_from_func(fn_get_y, classes=codes))
src.transform(tfms, tfm_y=True, size=config.LEARNER.img_size)
data = src.databunch(cocodata_path,
                     bs=config.LEARNER.bs,
                     val_bs=2*config.LEARNER.bs,
                     num_workers = config.LEARNER.num_workers)
stats = [torch.tensor([0.2190, 0.1984, 0.1928]), torch.tensor([0.0645, 0.0473, 0.0434])]
data.normalize(stats);
data.c_in, data.c_out = 3, 2

### For converting Validation set into a mini set to experiment on

In [12]:
fn_get_y = lambda image_name: Path(image_name).parent.parent/('labels/'+Path(image_name).name)
codes = np.loadtxt(cocodata_path/'codes.txt', dtype=str)
tfms = get_transforms()
src = (SegCustomItemList
       .from_folder(test_path, recurse=True, extensions='.jpg')
       .filter_by_func(lambda fname: Path(fname).parent.name == 'images', )
       .split_by_rand_pct(0.9)
       .label_from_func(fn_get_y, classes=codes))
src.transform(tfms, tfm_y=True, size =config.LEARNER.img_size)
data = src.databunch(test_path,
                     bs=config.LEARNER.bs,
                     val_bs=2*config.LEARNER.bs,
                     num_workers = config.LEARNER.num_workers)
stats = [torch.tensor([0.2190, 0.1984, 0.1928]), torch.tensor([0.0645, 0.0473, 0.0434])]
data.normalize(stats);
data.c_in, data.c_out = 3, 2

## Blur

In [13]:
from model import GaussianSmoothing

In [14]:
gg = GaussianSmoothing(2, 3, 2)

In [15]:
gg(torch.rand(1,2,256,256)).shape

torch.Size([1, 2, 254, 254])

In [16]:
deepCap = CapsNet(config.MODEL).cuda()

In [17]:
%%time
deepCap(torch.rand(1,3,256,256).cuda())

torch.Size([1, 512, 8, 8])
torch.Size([1, 512, 6, 6])
torch.Size([1, 512, 8, 8])
torch.Size([1, 128, 16, 16])
torch.Size([1, 128, 14, 14])
torch.Size([1, 128, 16, 16])
torch.Size([1, 16, 32, 32])
torch.Size([1, 16, 30, 30])
torch.Size([1, 16, 32, 32])
torch.Size([1, 128, 64, 64])
torch.Size([1, 128, 62, 62])
torch.Size([1, 128, 64, 64])
torch.Size([1, 16, 128, 128])
torch.Size([1, 16, 126, 126])
torch.Size([1, 16, 128, 128])
CPU times: user 126 ms, sys: 51.5 ms, total: 177 ms
Wall time: 178 ms


tensor([[[[0.0192, 0.0187, 0.0189,  ..., 0.0187, 0.0190, 0.0187],
          [0.0188, 0.0191, 0.0185,  ..., 0.0191, 0.0184, 0.0185],
          [0.0189, 0.0185, 0.0188,  ..., 0.0185, 0.0189, 0.0187],
          ...,
          [0.0184, 0.0191, 0.0183,  ..., 0.0193, 0.0182, 0.0185],
          [0.0189, 0.0187, 0.0188,  ..., 0.0186, 0.0189, 0.0187],
          [0.0184, 0.0183, 0.0184,  ..., 0.0186, 0.0184, 0.0183]],

         [[0.0225, 0.0208, 0.0223,  ..., 0.0209, 0.0223, 0.0212],
          [0.0223, 0.0226, 0.0222,  ..., 0.0224, 0.0222, 0.0217],
          [0.0223, 0.0208, 0.0223,  ..., 0.0209, 0.0223, 0.0210],
          ...,
          [0.0222, 0.0222, 0.0222,  ..., 0.0222, 0.0222, 0.0215],
          [0.0223, 0.0207, 0.0223,  ..., 0.0210, 0.0222, 0.0211],
          [0.0216, 0.0217, 0.0218,  ..., 0.0217, 0.0218, 0.0215]]]],
       device='cuda:0', grad_fn=<SqrtBackward>)

### seg_model Caps

In [13]:
run_dir = config.LEARNER.runsave_dir+'/'+name
os.makedirs(run_dir, exist_ok=True)
exp_name = 'seg_model_caps'
mlflow_CB = partial(MLFlowTracker,
                    exp_name=exp_name,
                    uri='file:/workspace/oct_ca_seg/runsaves/fastai_experiments/mlruns/',
                    params=config.config_dict,
                    log_model=True,
                    nb_path="/workspace/oct_ca_seg/oct/02_caps.ipynb")
deepCap = CapsNet(config.MODEL).cuda()
learner = Learner(data = data,
                  model = deepCap,
                  metrics = metrics,
                  callback_fns=mlflow_CB)

In [ ]:
with mlflow.start_run():
    learner.fit_one_cycle(config.LEARNER.epochs, slice(config.LEARNER.lr), pct_start=0.9)
    MLPY.save_model(learner.model, run_dir+'/model')
    save_all_results(learner, run_dir, exp_name)
    saveConfigRun(config.config_dict, run_dir=Path(run_dir), name = 'configCAPS_APPresnet18_bs16_epochs15_lr0.001.json')